In [1]:
from datasets import load_from_disk, Array2D
import torch
import numpy as np
import time
from collections import Counter


In [2]:
ds = load_from_disk("../datasets_cache/v3-fsl-105-v3")
ds


DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'category', 'file', 'pose', 'face', 'lh', 'rh', 'pose_mean', 'face_mean', 'lh_mean', 'rh_mean', 'fps', 'keypoints_length'],
        num_rows: 1755
    })
    test: Dataset({
        features: ['id', 'label', 'category', 'file', 'pose', 'face', 'lh', 'rh', 'pose_mean', 'face_mean', 'lh_mean', 'rh_mean', 'fps', 'keypoints_length'],
        num_rows: 196
    })
})

In [9]:
pose = ds["train"][0]["pose"][0]
print(np.mean(pose[0::4]))
print(np.mean(pose[1::4]))
print(np.mean(pose[2::4]))
# test = torch.concat([pose, face, rh, lh])
# print(test.shape)

-1.045974027929884e-09
-3.83817788326379e-09
2.5964144504431524e-09


In [3]:
train_ctr = Counter(ds['train']['label']).most_common()
test_ctr = Counter(ds['test']['label']).most_common()
feature = ds["train"].features['label']


train_ctr = list(map(lambda item: (feature.int2str(item[0]), item[1]), train_ctr))
test_ctr = list(map(lambda item: (feature.int2str(item[0]), item[1]), test_ctr))

print(train_ctr)
print(test_ctr)

[('IDLE', 86), ('NICE TO MEET YOU', 20), ('GOOD EVENING', 20), ('SEVEN', 19), ('WRONG', 19), ('NO', 19), ('SUGAR', 19), ('FAST', 19), ('KNOW', 19), ('COLD', 19), ('JUICE', 19), ('GOOD AFTERNOON', 19), ('HOW ARE YOU', 19), ('GRAY', 18), ('TEN', 18), ('SEE YOU TOMORROW', 18), ('AUNTIE', 18), ('UNCLE', 18), ('BROWN', 18), ('YES', 18), ('WINE', 18), ('THANK YOU', 18), ('VIOLET', 18), ('MILK', 18), ('CHICKEN', 18), ('CRAB', 18), ('LONGANISA', 18), ('MAN', 18), ('RICE', 18), ('HOT', 18), ('SLOW', 18), ('THURSDAY', 18), ('MEAT', 18), ('SUNDAY', 18), ('TUESDAY', 18), ('TEA', 18), ('BREAD', 18), ('MONDAY', 18), ('COFFEE', 18), ('NO SUGAR', 18), ('SHRIMP', 18), ('SATURDAY', 18), ('SPAGHETTI', 18), ('PINK', 17), ('COUSIN', 17), ('EIGHT', 17), ('FISH', 17), ('BLACK', 17), ('SIX', 17), ('DON’T KNOW', 17), ('BEER', 17), ('EGG', 17), ('DEAF BLIND', 17), ('GOOD MORNING', 17), ('GREEN', 16), ('DEAF', 16), ('THREE', 16), ('WHITE', 16), ('MARRIED', 16), ('NINE ', 16), ('DECEMBER', 16), ('AUGUST', 16), ('

In [4]:
train_ctr = Counter(ds['train']['keypoints_length']).most_common()
test_ctr = Counter(ds['test']['keypoints_length']).most_common()

print(train_ctr)
print(test_ctr)

[(20, 176), (21, 171), (19, 159), (23, 147), (22, 146), (17, 144), (18, 143), (24, 111), (25, 88), (16, 82), (26, 74), (27, 73), (15, 69), (28, 47), (29, 35), (30, 24), (31, 21), (32, 17), (33, 13), (34, 5), (35, 4), (43, 2), (37, 1), (38, 1), (39, 1), (44, 1)]
[(19, 21), (18, 18), (20, 18), (23, 18), (24, 17), (17, 15), (22, 15), (21, 14), (15, 9), (16, 9), (25, 8), (26, 8), (27, 7), (29, 7), (30, 5), (28, 3), (31, 2), (32, 1), (36, 1)]


In [3]:
ds["train"].features
# ds["train"].flatten()

{'id': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['A', 'APRIL', 'AUGUST', 'AUNTIE', 'B', 'BEER', 'BLACK', 'BLIND', 'BLUE', 'BOY', 'BREAD', 'BROWN', 'C', 'CHICKEN', 'COFFEE', 'COLD', 'CORRECT', 'COUSIN', 'CRAB', 'D', 'DARK', 'DAUGHTER', 'DEAF', 'DEAF BLIND', 'DECEMBER', 'DON’T KNOW', 'DON’T UNDERSTAND', 'DRINK', 'E', 'EAT', 'EGG', 'EIGHT', 'F', 'FAST', 'FATHER', 'FEBRUARY', 'FISH', 'FIVE', 'FOUR', 'FRIDAY', 'G', 'GIRL', 'GOOD AFTERNOON', 'GOOD EVENING', 'GOOD MORNING', 'GRANDFATHER', 'GRANDMOTHER', 'GRAY', 'GREEN', 'H', 'HARD OF HEARING', 'HELLO', 'HOT', 'HOW ARE YOU', 'HOW OLD ARE YOU', 'I', 'I (ME)', 'IDLE', 'IM FINE', 'IM SORRY', 'J', 'JANUARY', 'JUICE', 'JULY', 'JUNE', 'K', 'KNOW', 'L', 'LIGHT', 'LONGANISA', 'M', 'MAN', 'MARCH', 'MARRIED', 'MAY', 'MEAT', 'MILK', 'MINE', 'MONDAY', 'MOTHER', 'MY', 'N', 'NICE TO MEET YOU', 'NINE ', 'NO', 'NO SUGAR', 'NOVEMBER', 'O', 'OCTOBER', 'ONE', 'ORANGE', 'P', 'PARENTS', 'PINK', 'Q', 'R', 'RED', 'RICE', 'S', 'SATURDAY', 'SEE YOU TO

In [10]:
data = ds["train"].filter(lambda x: x["keypoints_length"] > 150, num_proc=4)

In [13]:
data.remove_columns("keypoints")
data = data.sort("keypoints_length", reverse=True)
data["file"]

['../Dataset/fsl-143-v1\\clips/125\\7.mp4',
 '../Dataset/fsl-143-v1\\clips/123\\8.mp4',
 '../Dataset/fsl-143-v1\\clips/125\\8.mp4',
 '../Dataset/fsl-143-v1\\clips/124\\8.mp4',
 '../Dataset/fsl-143-v1\\clips/121\\13.mp4',
 '../Dataset/fsl-143-v1\\clips/131\\12.mp4',
 '../Dataset/fsl-143-v1\\clips/131\\13.mp4',
 '../Dataset/fsl-143-v1\\clips/122\\7.mp4',
 '../Dataset/fsl-143-v1\\clips/134\\7.mp4',
 '../Dataset/fsl-143-v1\\clips/139\\10.mp4',
 '../Dataset/fsl-143-v1\\clips/121\\7.mp4',
 '../Dataset/fsl-143-v1\\clips/134\\12.mp4',
 '../Dataset/fsl-143-v1\\clips/140\\10.mp4',
 '../Dataset/fsl-143-v1\\clips/122\\12.mp4',
 '../Dataset/fsl-143-v1\\clips/130\\10.mp4',
 '../Dataset/fsl-143-v1\\clips/140\\12.mp4',
 '../Dataset/fsl-143-v1\\clips/138\\13.mp4',
 '../Dataset/fsl-143-v1\\clips/126\\12.mp4',
 '../Dataset/fsl-143-v1\\clips/135\\11.mp4',
 '../Dataset/fsl-143-v1\\clips/134\\11.mp4',
 '../Dataset/fsl-143-v1\\clips/124\\12.mp4',
 '../Dataset/fsl-143-v1\\clips/138\\7.mp4',
 '../Dataset/fsl-1

In [87]:
max_jitter = torch.tensor([0.1, 0.2, 0.3])
random = torch.rand(3) 
output = random * max_jitter - (max_jitter / 2)

print(random)
print(output)

True if torch.rand(1) < 0.5 else False

tensor([0.0808, 0.6036, 0.2923])
tensor([-0.0419,  0.0207, -0.0623])


False